# Transfer learning и ResNet
 В этот раз мы продолжим работать с изображениями с помощью нейронных сетей. В этот раз мы обсудим, как адаптировать уже существующие обученные модели для своих задач

План:
- ResNet
- transfer learning и fine-tuning
- Визуализация модели
- Torchmetrics, scheduler, ???

In [ ]:
import glob
import warnings

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchmetrics
import tqdm
from torch import nn
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Normalize, Resize, ToTensor

%matplotlib inline
warnings.filterwarnings("ignore")

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
from utils.common import seed_everything
seed_everything(123)

In [ ]:
DATA_PATH = "D:\data\spbu_dl\genshin"
# TODO: change path to yours!

In [ ]:
dataset = ImageFolder(
    DATA_PATH,
    transform=Compose(
        [
            Resize((224, 224)),
            ToTensor(),
            Normalize((0.5, 0.5, 0.5), (1, 1, 1)),
        ]
    ),
)
train_set, val_set = torch.utils.data.random_split(
    dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))]
)

train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=16, shuffle=False)

*Задание*: выведите случайный элемент датасета

In [ ]:
# < YOUR CODE >

In [ ]:
# выведите 352 семпл

Тут мы сталкиваемся с двумя проблемами сразу:
- датасет очень маленький, и собирать его руками достаточно сложно
- если мы попробуем обучить какую-то модель с нуля, это может занять очень много времени

К тому же, наши картинки еще довольно похожи на стандартные изображения. Поэтому можно попытаться как-то перенести уже полученные знания какой-то крутой модели на свою задачу.
Чаще всего для этого ипользуется файн-тюнинг.

Идея у него проста: модель, когда-то обученная на большом датасете, уже умеет выделять какие-то хорошие признаки из входных данных, поэтому мы можем взять ее веса и немного поменять их, чтобы она лучше понимала новую задачу.


## ResNet
Исследователи заметили, что когда речь идет о сверточных нейронных сетях, чем модель глубже, тем лучше. Это имеет смысл, потому что их гибкость в адаптации к любому пространству увеличивается с увеличением числа параметров). Однако было замечено, что начиная с некоторй глубины, точность перестает увеличиваться, а наоборот, начинает падать.
*Вопрос:* с чем связано это явление?

Так появилось семейство ResNet. Идея проста: еслт модель может предсказать $f(x)$, то может предсказать и $f(x) - x$.
Это очень полезно, так как позволяет улучшить обучаемость моделей. (Чем?)
![alt_text](../additional_materials/images/resnet.png)

В модели это реализовано следующим образом: к выходу слоя можно добавить его же вход (что распространяется и на более длинные связи).

Обычный блок выглядит примерно следующим образом:

Input: x

[Conv 3x3]
[Batch Normalization]
[Activation (e.g., ReLU)]
[Conv 3x3]
[Batch Normalization]

Add: x + F(x) (Output from the above layers)
[Activation (e.g., ReLU)]


Глубокие варианты используют не двухслойные блоки, но также и bottleneck блоки, которые состоят из трех сверточных слоев следующего вида:

[Conv 1x1]
[BN + ReLU]
[Conv 3x3]
[BN + ReLU]
[Conv 1x1]
[BN]

*Вопрос*: Почему используются блоки 3x3?
*Вопрос 2*: А почему по два таких слоя в блоке, а не один?


В библиотеке torchvision имплементировано не только большое множество моделей (всевозможные ResNet'ы, Inception, VGG, AlexNet, DenseNet, ResNext, WideResNet, MobileNet...), но и загружены чекпоинты обучения этих моделей на ImageNet.

In [ ]:
from torchvision.models import resnet18

In [ ]:
model = resnet18(pretrained=True)

In [ ]:
# выведите модель

## Transfer learning
В задаче transfer learning'a последний слой нейросети заменяется на линейный с нужным числом выходов, а веса остальных слоёв "замораживаются".
Иногда делают полный файнтюн, тогда основные веса тоже обучаются, но в таком случае нужно задавать очень маленькие значения скорости обучения, чтобы не забыть то, что модель выучила раньше

Задание: определите, сколько классов в нашем датасете

In [ ]:
# YOUR CODE

Теперь научимся загружать веса нашей модели, а также переделывать ее под свои нужды.

In [ ]:
def get_model(
        pretrained: bool = True, num_out_classes: int=10
):
    if pretrained:
        model = resnet18(pretrained=True)
        # freeze parameters of original model
        for param in model.parameters():
            param.requires_grad = False
        model.fc = nn.Linear(model.fc.in_features, num_out_classes)
    else:
        # <YOUR CODE>
    return model


Проверим нашу функцию

In [ ]:
model = get_model(pretrained=True, num_out_classes=5)

In [ ]:
optimizer = # задайте оптимизатор

In [ ]:
# Проверьте, задан ли градиент у какого-нибудь слоя из тела модели (это все, что не голова)

In [ ]:
loss = # задайте подходящий лосс

В этом ноутбуке мы рассмотрим еще очень полезную библиотеку, которая поможет нам считать метрики качества.

In [ ]:
import torchmetrics
metric = torchmetrics.Accuracy(
    task="multiclass",
    num_classes=5
)

*Задание*: добавьте метрики Precision, recall.
*Задание 2*: сравните метрики с reduction=micro, macro. Ответьте, в чем разница и в каких ситуациях та или иная настройка лучше отражает качество предсказаний.

Вспомним рутину: напишем обучающий цикл. Заполните функциии в нужных местах.

In [ ]:
def train_epoch(
    model: nn.Module,
    data_loader: torch.utils.data.DataLoader,
    criterion: nn.Module,
    metric: torchmetrics.Metric,
    optimizer: torch.optim.Optimizer,
    return_losses=False,
):
    model = model.train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    with tqdm.tqdm(total=len(data_loader)) as prbar:
        for inputs, targets in data_loader:
            # train step: preds, loss, optimization, etc
            # <YOUR CODE>
            # update description for tqdm
            accuracy = metric(preds, targets)
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            num_batches += 1
            all_losses.append(loss.detach().item())
    acc = metric.compute()
    metrics = {"loss": total_loss / num_batches, "accuracy": acc}
    if return_losses:
        return metrics, all_losses
    else:
        return metrics


def validate(
        model,
        data_loader,
        criterion,
        metric,
        device="cuda:0"):
    model = model.eval()
    total_loss = 0
    num_batches = 0
    with tqdm.tqdm(total=len(data_loader)) as prbar:
        for inputs, targets in data_loader:
            with torch.no_grad():
                # <YOUR CODE>
                accuracy = # <YOUR CODE>
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            num_batches += 1
    metrics = # <YOUR CODE>
    return metrics

Иногда удобно сохранять данные в определенном формате. Один из возможных вариантов - namedtuple. Он похволяет как сохранять порядок элементов, так и использовать более понятную индексацию

In [ ]:
from collections import namedtuple

LossInfo = namedtuple(
    "LossInfo", ["full_train_losses", "train_epoch_losses", "eval_epoch_losses"]
)
EPOCHS = 10
LR = 0.001

In [ ]:
def train(
    model,
    epochs,
    train_data_loader,
    validation_data_loader,
    criterion,
    metric,
    optimizer,
    scheduler=None,
    device="cuda:0",
):
    all_train_losses = []
    epoch_train_losses = []
    epoch_eval_losses = []
    for epoch in range(epochs):
        # construct iterators
        train_iterator = iter(train_data_loader)
        validation_iterator = iter(validation_data_loader)
        # train step
        print(f"Train Epoch: {epoch}")
        train_metrics, one_epoch_train_losses = train_epoch(
            model=model,
            data_loader=train_iterator,
            criterion=criterion,
            metric=metric,
            optimizer=optimizer,
            return_losses=True,
        )
        # save train losses
        all_train_losses.extend(one_epoch_train_losses)
        epoch_train_losses.append(train_metrics["loss"])
        # eval step
        print(f"Validation Epoch: {epoch}")

        # call function validation epoch and
        # save eval losses
        # <YOUR CODE>
        # scheduler step
        if scheduler:
            scheduler.step()
    return LossInfo(all_train_losses, epoch_train_losses, epoch_eval_losses)

In [ ]:
model.to(device)
_ = train(
    model=model,
    epochs=5,
    train_data_loader=train_dataloader,
    validation_data_loader=val_dataloader,
    criterion=loss,
    metric=metric,
    optimizer=torch.optim.SGD(model.parameters(), lr=LR),
    device=device,
)

*Задание*: запустите обучение с использованием transfer learning и без него. В каком случае точность получилась выше?


# Визуализация сети

Каждый слой в CNN изучает фильтры возрастающей сложности. На первых слоях выучиваются простые признаки, такие как края и углы. Средние слои обнаруживают части объектов — что касается лиц, они могут научиться реагировать на глаза и носы. Последние слои имеют более высокоуровневые представления: они учатся распознавать объекты целиком, в различных формах и положениях.

Мы попробуем проверить, так ли это, с помощью GradientExplainer, который использует ожидаемые градиенты для оценки входов в разные части модели. В целом, они аппроксимируют значения SHAP.

Возьмем по 50 семплов для подсчёта ожидаемых градиентов и посмотрим на признаки первых слоёв ResNet.

In [ ]:
model.eval()
data = dataset[0]


In [ ]:
dataset.classes

In [ ]:
import shap
to_explain = torch.stack([dataset[0][0], dataset[123][0]], dim=0)
e = shap.GradientExplainer((model, model.layer1[0].conv2), to_explain)
# receive shap_values
shap_values, indexes = e.shap_values(
    to_explain, ranked_outputs=4, nsamples=50
)

# plot the explanations
#shap_values = [np.swapaxes(np.swapaxes(s, 2, 3), 1, -1) for s in shap_values]


In [ ]:
shap_values.shape

In [ ]:
shap.image_plot(shap_values[0], to_explain.detach().cpu().numpy())

In [ ]:
shap.image_plot(shap_values[1], to_explain.detach().cpu().numpy())

Повторите для второго слоя

In [ ]:
# YOUR CODE

In [ ]:
# YOUR CODE

Повторите для 4 слоя)

In [ ]:
# YOUR CODE

Но это не единственный способ заглянуть внутрь модели. Простейший вариант - оценить значения активаций на внутренних слоях. Можно было бы вывести веса, но они, к сожалению, не слишком интерпретируемы. Активации же показывают силу реакции конкретного фильтра.

Далее попробуем посмотреть на значения активаций при при проходе через изображение. В этом нам помогут хуки - эти функции позволяют получить доступ к состоянию между слоями (как входа, так и выхода слоя)
для визуализации, трансфер-лернинга, дебага модели итд.

In [ ]:
activations = {}

# Function to create a hook
def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

# For example, let's hook into the first convolutional layer and the first block of layer1
model.conv1.register_forward_hook(get_activation('conv1'))
model.layer1[0].register_forward_hook(get_activation('layer1_block1'))


In [ ]:
# Function to visualize feature maps
def visualize_feature_maps(activation, title, num_cols=8):
    num_feature_maps = activation.shape[0]
    num_rows = num_feature_maps // num_cols
    plt.figure(figsize=(num_cols, num_rows))
    for i in range(num_feature_maps):

        if i >= num_cols * num_rows:
            break
        plt.subplot(num_rows, num_cols, i+1)
        plt.imshow(activation[i].cpu().numpy(), cmap='coolwarm')
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

#

Выведите значения активаций

In [ ]:
# Perform a forward pass
with torch.no_grad():
    # прогоните модель для первого элемента датасета
    output = model(#your code)


In [ ]:
for name, activation in activations.items():
    # For convolutional layers, activation shape is (batch_size, num_channels, H, W)
    act = activation[0]
    act = act - act.min() # Почему нужно нормализовать?
    act = act / act.max()
    visualize_feature_maps(act, title=f'Activations from {name}')

In [ ]:
# попробуйте добавить хук и для 4 слоя. Какие значения у активаций? Похоже ли это на то, что мы видели для shap values?